### Import Libraries 

In [72]:


import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline 
import xgboost as xgb

#Need to clean further 
from Standardization import metric_normalizer
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler,OneHotEncoder, OrdinalEncoder

from sklearn.metrics import confusion_matrix,accuracy_score, classification_report  # Import accuracy_score


from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,StandardScaler, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA 
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.feature_selection import SelectPercentile, mutual_info_classif
from sklearn.impute import SimpleImputer

from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict

from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, f1_score, precision_score, recall_score

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.exceptions import UndefinedMetricWarning

# Filter out UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)



### Import Dataset

In [2]:
df = pd.read_csv('/home/clivence/base_jupyter/Datadump/Model_Data/GU_Model_Data_V1_16k')

### Data Cleaning

In [3]:
#Filter out negative values - evetually need to research why this is happening 
df  = df[(df['2D Low in Pips'] > 0) | (df['2D Low in Pips'] > 0)] 

df = df[df['Action'] == 'Ultimate Action']
df = df[df['Ticker'] == 'GBP/USD']

df.fillna(0, inplace=True)

#Standardize the dataset
df =  metric_normalizer(df)

#Set date to datetime 
df['Date'] = pd.to_datetime(df['Date'])

#Create Date Derived Features 
df['Trade_Week_Year'] = df['Date'].dt.isocalendar().week
df['Trade_Week_Month'] = (df['Date'].dt.day -1)//7+1
df['Trade_Day_Week'] = df['Date'].dt.weekday + 1

#Create a new feature to identify the status of the previous trade 
df['Previous_Trade_Status'] = df['2D Trade Status'].shift(fill_value=0)


### Data Preprocessing 

In [59]:
#Subset Selection
df = df[['open', 'high', 'low', 'close', 'volume','Trade_Week_Year','Trade_Week_Month','Trade_Day_Week',
       'Day','Month','25EMA', '50EMA', '75EMA','100EMA', '125EMA',
        'Trend Status', 'Spread','5075 Trend Status', '75100 Trend Status',
       '100125 Trend Status', 'Order Type', 
        '%K', '%D', 'k_group', 'ADX', 'ADXR', 'slowk','slowd','CDL2CROWS', 'CDL3BLACKCROWS',
        'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE',
        'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK',
        'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL',
        'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI',
        'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE',
        'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS',
        'CDLHIGHWAVE', 'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS',
        'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLKICKINGBYLENGTH', 'CDLLADDERBOTTOM',
        'CDLLONGLEGGEDDOJI', 'CDLLONGLINE', 'CDLMARUBOZU', 'CDLMATCHINGLOW', 'CDLMATHOLD',
        'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR', 'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN',
        'CDLRISEFALL3METHODS', 'CDLSEPARATINGLINES', 'CDLSHOOTINGSTAR', 'CDLSHORTLINE',
        'CDLSPINNINGTOP', 'CDLSTALLEDPATTERN', 'CDLSTICKSANDWICH', 'CDLTAKURI', 'CDLTASUKIGAP',
        'CDLTHRUSTING', 'CDLTRISTAR', 'CDLUNIQUE3RIVER', 'CDLUPSIDEGAP2CROWS', 'CDLXSIDEGAP3METHODS',
        'candle_bullish_score','candle_bearish_score','Previous_Trade_Status','2D Trade Status']]

#Set a random state 
state =42

In [5]:
# set training, test, and validation set size 
total_len = len(df)
training_size = int(total_len * .8)
test_size = int(total_len * .1)
validation_size = total_len - training_size - test_size

# Split indices
training_idx = df.index[:training_size]
test_idx = df.index[training_size:training_size + test_size]
validation_idx = df.index[-validation_size:]

# Verify lengths
assert len(training_idx) == training_size
assert len(test_idx) == test_size
assert len(validation_idx) == validation_size

# Create training, test, and validation df 
training_df = df.loc[training_idx]
test_df = df.loc[test_idx]
validation_df = df.loc[validation_idx]

# Print lengths of the three DataFrames
print(f"Training: {len(training_df)}")
print(f"Test: {len(test_df)}")
print(f"Validation: {len(validation_df)}")

Training: 386
Test: 48
Validation: 49


In [6]:
X_train = training_df.drop(columns=['2D Trade Status'])
y_train = training_df['2D Trade Status']
X_test = test_df.drop(columns=['2D Trade Status'])
y_test = test_df['2D Trade Status']

In [7]:
#Create a list of columns to encode 
cat_cols_to_encode = ['Order Type']
#Create a list of cols for ordinal encoding 
cat_cols_for_ordinal_encoding = ['Trend Status','5075 Trend Status', '75100 Trend Status',
                    '100125 Trend Status','k_group']

#Create a list of cols to scale 
num_cols_to_scale = ['volume']

# Create Column Transformer with appropriate preprocessing for numerical and categorical data
Preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
            ('scaler', MinMaxScaler())]), num_cols_to_scale),
        ('cat_onehot', OneHotEncoder(drop='first', sparse_output=False), cat_cols_to_encode),
        ('cat_ordinal', OrdinalEncoder(), cat_cols_for_ordinal_encoding)
    ],
    remainder='passthrough')

# Preprocessor.set_output(transform='pandas')

## Base Models

### Logistic Regression

In [73]:
#Create PiPeline to be used in Model
Base_LR_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('RF_Model',LogisticRegression())])

# Perform cross-validation
Base_LR_Model_Pipeline.fit(X_train, y_train)
Base_LR_Model_Predictions = Base_LR_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, Base_LR_Model_Predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate and Print  classification report
report = classification_report(y_test, Base_LR_Model_Predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Test Accuracy: 0.6458333333333334

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.00      0.00      0.00        13
  Win Trades       0.70      0.89      0.78        35

    accuracy                           0.65        48
   macro avg       0.35      0.44      0.39        48
weighted avg       0.51      0.65      0.57        48



/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Logistic Regression Summary and Key Takeaways

### Performance Overview
- **Test Accuracy**: Approximately 64.6%.

### Insights from Classification Report
- **'Win Trades' (Positive Class)**:
  - Precision: ~70%
  - Recall: ~89%
  - F1-score: ~0.78
- **'Loss Trades' (Negative Class)**:
  - Precision: 0% (no correct predictions for this class)
  - Recall: 0%
  - F1-score: 0.00

### Key Takeaways
1. **Limited Performance on Minority Class**:
   - The model performs reasonably well in predicting 'Win Trades' but fails to correctly identify 'Loss Trades', leading to significant class imbalance issues.
2. **Imbalance Remediation Needed**:
   - Addressing the class imbalance through techniques like adjusting class weights, resampling, or using different algorithms is critical to improving overall model performance and fairness.
3. **Evaluation Beyond Accuracy**:
   - While accuracy is an essential metric, it fails to capture the model's performance accurately in the presence of class imbalance. Metrics like precision, recall, and the F1-score provide a more comprehensive understanding of the model's capabilities.


### Support Vector Classifier 

In [74]:
#Create Pipeline to be used in SVC Model
Base_SVC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('SVC_Model',SVC(random_state=state))])

#Fit the training data 
Base_SVC_Model_Pipeline.fit(X_train, y_train)

#Make Predictions on the test set 
Base_SVC_Model_Predictions = Base_SVC_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, Base_SVC_Model_Predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate  and Print a classification report
report = classification_report(y_test, Base_SVC_Model_Predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Test Accuracy: 0.7291666666666666

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.00      0.00      0.00        13
  Win Trades       0.73      1.00      0.84        35

    accuracy                           0.73        48
   macro avg       0.36      0.50      0.42        48
weighted avg       0.53      0.73      0.61        48



## Support Vector Machine Summary and Key Takeaways

### Performance Overview
- **Test Accuracy**: Approximately 71-73%.

### Insights from Classification Report
- **'Win Trades' (Positive Class)**:
  - Precision: ~73%
  - Recall: ~94%
  - F1-score: ~0.82
- **'Loss Trades' (Negative Class)**:
  - Precision: Varies, often low (0-33%)
  - Recall: Often low (0-8%)
  - F1-score: Varies, often low (0-0.12)

### Key Takeaways
1. **Imbalance in Class Performance**:
   - Model excels in predicting 'Win Trades' but struggles with 'Loss Trades', particularly in recall.
2. **Model Optimization for Minority Class**:
   - Further optimization needed to improve prediction of 'Loss Trades'. Consider techniques like adjusting class weights or oversampling.
3. **Consider Data Quality and Representation**:
   - Ensure test set is representative. Investigate reasons for low performance in predicting 'Loss Trades'.
4. **Iterative Model Refinement**:
   - Experiment with different algorithms, feature engineering, and hyperparameter tuning to achieve better balance between precision and recall for both classes.


### Decision Tree Classifier 

In [60]:
#Create Pipeline to be used in Descision Tree Model
Base_DT_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('DT_Model',DecisionTreeClassifier(random_state=state))])

#Fit the training data 
Base_DT_Model_Pipeline.fit(X_train, y_train)

#Make Predictions on the test set 
Base_DT_Model_Predictions = Base_DT_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, Base_DT_Model_Predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate  and Print a classification report
report = classification_report(y_test, Base_DT_Model_Predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Test Accuracy: 0.6041666666666666

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.25      0.23      0.24        13
  Win Trades       0.72      0.74      0.73        35

    accuracy                           0.60        48
   macro avg       0.49      0.49      0.49        48
weighted avg       0.59      0.60      0.60        48



## Decision Tree Classifier Summary and Key Takeaways

### Performance Overview
- **Test Accuracy**: Approximately 60.4%.

### Insights from Classification Report
- **'Win Trades' (Positive Class)**:
  - Precision: ~72%
  - Recall: ~74%
  - F1-score: ~0.73
- **'Loss Trades' (Negative Class)**:
  - Precision: ~25%
  - Recall: ~23%
  - F1-score: ~0.24

### Key Takeaways
1. **Imbalance in Class Performance**:
   - Model performs better in predicting 'Win Trades' compared to 'Loss Trades'.
2. **Model Optimization**:
   - Consider model parameter tuning or alternative algorithms to improve overall performance.
3. **Interpretability**:
   - Decision trees provide interpretable models, but their performance may vary based on the dataset and its characteristics.


### ADA Boost Classifier

In [66]:
#Create PiPeline to be used in Model
Base_ADAC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('RF_Model',AdaBoostClassifier(random_state=state))])

#Fit the training data 
Base_ADAC_Model_Pipeline.fit(X_train, y_train)

#Make Predictions on the test set 
Base_ADAC_Model_Predictions = Base_ADAC_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, Base_ADAC_Model_Predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate  and Print a classification report
report = classification_report(y_test, Base_ADAC_Model_Predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)

/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Test Accuracy: 0.6875

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.25      0.08      0.12        13
  Win Trades       0.73      0.91      0.81        35

    accuracy                           0.69        48
   macro avg       0.49      0.50      0.46        48
weighted avg       0.60      0.69      0.62        48



## ADA Boost Classifier Summary and Key Takeaways

### Performance Overview
- **Test Accuracy**: Approximately 68.8%.

### Insights from Classification Report
- **'Win Trades' (Positive Class)**:
  - Precision: ~73%
  - Recall: ~91%
  - F1-score: ~0.81
- **'Loss Trades' (Negative Class)**:
  - Precision: ~25%
  - Recall: ~8%
  - F1-score: ~0.12

### Key Takeaways
1. **Imbalance in Class Performance**:
   - Model performs significantly better in predicting 'Win Trades' compared to 'Loss Trades'.
2. **Model Optimization**:
   - Consider adjusting model parameters or exploring different algorithms to improve performance, particularly for the minority class.
3. **Data Imbalance Consideration**:
   - Addressing class imbalance may help improve the overall performance and reliability of the model.


### Random Forest Classifier

In [67]:
# Create Model Pipeline
Base_RF_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('RF_Model', RandomForestClassifier(random_state=state))  
])

#Fit the training data 
Base_RF_Model_Pipeline.fit(X_train, y_train)

# Make Predictions
RF_CV_predictions = Base_RF_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, RF_CV_predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate  and Print a classification report
report = classification_report(y_test, RF_CV_predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Test Accuracy: 0.7291666666666666

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.00      0.00      0.00        13
  Win Trades       0.73      1.00      0.84        35

    accuracy                           0.73        48
   macro avg       0.36      0.50      0.42        48
weighted avg       0.53      0.73      0.61        48



/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

## Random Forest Classifier Summary and Key Takeaways

### Performance Overview
- **Test Accuracy**: Approximately 72.9%.

### Insights from Classification Report
- **'Win Trades' (Positive Class)**:
  - Precision: ~73%
  - Recall: 100%
  - F1-score: ~0.84
- **'Loss Trades' (Negative Class)**:
  - Precision: 0% (no correct predictions for this class)
  - Recall: 0%
  - F1-score: 0.00

### Key Takeaways
1. **Severe Class Imbalance Impact**:
   - The model predicts 'Win Trades' with high recall but completely fails to identify 'Loss Trades', indicating a possible overfitting to the majority class.
2. **Urgent Need for Model Calibration**:
   - It is crucial to address the overfitting and improve the sensitivity towards the 'Loss Trades' by adjusting class weights, trying different algorithms, or employing techniques like resampling to balance the classes.
3. **Model Evaluation**:
   - This scenario underscores the importance of using metrics beyond accuracy, such as precision, recall, and the F1-score, to truly understand model performance, particularly in the presence of class imbalance.


### Gradient Boosting Classifier

In [70]:
#Create Pipeline to be used in Gradient Boosting Classfier Model
Base_GBC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('GBC_Model',GradientBoostingClassifier(random_state=state))])

#Fit the training data 
Base_GBC_Model_Pipeline.fit(X_train, y_train)

# Perform cross-validation
GBC_predictions = Base_GBC_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, GBC_predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate  and Print a classification report
report = classification_report(y_test, GBC_predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Test Accuracy: 0.7291666666666666

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.50      0.08      0.13        13
  Win Trades       0.74      0.97      0.84        35

    accuracy                           0.73        48
   macro avg       0.62      0.52      0.49        48
weighted avg       0.67      0.73      0.65        48



## Gradient Boosting Classifier Summary and Key Takeaways

### Performance Overview
- **Test Accuracy**: Approximately 72.9%.

### Insights from Classification Report
- **'Win Trades' (Positive Class)**:
  - Precision: ~74%
  - Recall: ~97%
  - F1-score: ~0.84
- **'Loss Trades' (Negative Class)**:
  - Precision: ~50%
  - Recall: ~8%
  - F1-score: ~0.13

### Key Takeaways
1. **Class Imbalance Impact**:
   - Model shows good performance in predicting 'Win Trades' but struggles with 'Loss Trades', as reflected in low recall for the negative class.
2. **Need for Class Balance**:
   - Address class imbalance to improve the model's ability to predict 'Loss Trades' accurately.
3. **Model Fine-Tuning**:
   - Explore parameter tuning or different algorithms to enhance overall model performance, particularly for minority classes.


In [77]:
#Create Pipeline to be used in Gradient Boosting Classfier Model
Base_XGB_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('XGB_Model',xgb.XGBClassifier(random_state=state))])

#Fit the training data 
Base_XGB_Model_Pipeline.fit(X_train, y_train)

# Perform cross-validation
XGB_predictions = Base_XGB_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, XGB_predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate  and Print a classification report
report = classification_report(y_test, XGB_predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Test Accuracy: 0.6875

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.25      0.08      0.12        13
  Win Trades       0.73      0.91      0.81        35

    accuracy                           0.69        48
   macro avg       0.49      0.50      0.46        48
weighted avg       0.60      0.69      0.62        48



## XGBoost Classifier Summary and Key Takeaways

### Performance Overview
- **Test Accuracy**: Approximately 68.75%.

### Insights from Classification Report
- **'Win Trades' (Positive Class)**:
  - Precision: ~73%
  - Recall: ~91%
  - F1-score: ~0.81
- **'Loss Trades' (Negative Class)**:
  - Precision: ~25%
  - Recall: ~8%
  - F1-score: ~0.12

### Key Takeaways
1. **Limited Ability in Predicting 'Loss Trades'**:
   - The model is able to predict 'Win Trades' fairly effectively, but shows very limited ability to correctly identify 'Loss Trades'.
2. **Improvement Needed in Negative Class Recognition**:
   - Significant improvements are needed in recognizing the negative class to reduce the model's bias towards 'Win Trades'.
3. **Potential Strategies for Enhancement**:
   - Consider revising the model parameters, using different feature selection techniques, or applying balanced class weighting strategies to better address class imbalance and improve overall model fairness.


## Cross Validate Top Models 

In [46]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

# Create Model Pipeline 
Base_SVC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor), 
    ('SVC_Model', SVC())
])

# Fit the model using cross-validation
cv_scores = cross_val_score(Base_SVC_Model_Pipeline, X_train, y_train, cv=5) 

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Fit the model on the entire training data
Base_SVC_Model_Pipeline.fit(X_train, y_train)

# Make predictions on the test set
Base_SVC_Model_Predictions = Base_SVC_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, Base_SVC_Model_Predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate a classification report
report = classification_report(y_test, Base_SVC_Model_Predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Cross-validation scores: [0.67948718 0.68831169 0.68831169 0.67532468 0.67532468]
Mean CV accuracy: 0.6813519813519813

Test Accuracy: 0.7291666666666666

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.00      0.00      0.00        13
  Win Trades       0.73      1.00      0.84        35

    accuracy                           0.73        48
   macro avg       0.36      0.50      0.42        48
weighted avg       0.53      0.73      0.61        48



/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

# Create Model Pipeline 
Base_SVC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor), 
    ('GBC_Model',GradientBoostingClassifier())
])

# Fit the model using cross-validation
cv_scores = cross_val_score(Base_SVC_Model_Pipeline, X_train, y_train, cv=5) 

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Fit the model on the entire training data
Base_SVC_Model_Pipeline.fit(X_train, y_train)

# Make predictions on the test set
Base_SVC_Model_Predictions = Base_SVC_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, Base_SVC_Model_Predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate a classification report
report = classification_report(y_test, Base_SVC_Model_Predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Cross-validation scores: [0.55128205 0.63636364 0.61038961 0.57142857 0.7012987 ]
Mean CV accuracy: 0.6141525141525142

Test Accuracy: 0.7291666666666666

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.50      0.08      0.13        13
  Win Trades       0.74      0.97      0.84        35

    accuracy                           0.73        48
   macro avg       0.62      0.52      0.49        48
weighted avg       0.67      0.73      0.65        48



In [48]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

# Create Model Pipeline 
Base_SVC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor), 
    ('RF_Model', RandomForestClassifier(random_state=42))
])

# Fit the model using cross-validation
cv_scores = cross_val_score(Base_SVC_Model_Pipeline, X_train, y_train, cv=5) 

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Fit the model on the entire training data
Base_SVC_Model_Pipeline.fit(X_train, y_train)

# Make predictions on the test set
Base_SVC_Model_Predictions = Base_SVC_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, Base_SVC_Model_Predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate a classification report
report = classification_report(y_test, Base_SVC_Model_Predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Cross-validation scores: [0.48717949 0.67532468 0.62337662 0.50649351 0.5974026 ]
Mean CV accuracy: 0.577955377955378

Test Accuracy: 0.7291666666666666

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.00      0.00      0.00        13
  Win Trades       0.73      1.00      0.84        35

    accuracy                           0.73        48
   macro avg       0.36      0.50      0.42        48
weighted avg       0.53      0.73      0.61        48



/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

# Create Model Pipeline 
Base_SVC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor), 
    ('XGB_Model',xgb.XGBClassifier())
])

# Fit the model using cross-validation
cv_scores = cross_val_score(Base_SVC_Model_Pipeline, X_train, y_train, cv=5) 

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Fit the model on the entire training data
Base_SVC_Model_Pipeline.fit(X_train, y_train)

# Make predictions on the test set
Base_SVC_Model_Predictions = Base_SVC_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, Base_SVC_Model_Predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate a classification report
report = classification_report(y_test, Base_SVC_Model_Predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Cross-validation scores: [0.6025641  0.61038961 0.61038961 0.50649351 0.51948052]
Mean CV accuracy: 0.5698634698634699

Test Accuracy: 0.6875

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.25      0.08      0.12        13
  Win Trades       0.73      0.91      0.81        35

    accuracy                           0.69        48
   macro avg       0.49      0.50      0.46        48
weighted avg       0.60      0.69      0.62        48



## Grid Search

### Random Forest Model

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to use for the grid search
param_grid = {
    'RF_Model__n_estimators': [25,50, 100, 200],  
    'RF_Model__criterion': ['gini', 'entropy'],  
    'RF_Model__max_depth': [None, 10, 20, 30],  
    'RF_Model__min_samples_split': [2, 5, 10],  
    'RF_Model__min_samples_leaf': [1, 2, 4], 
    'RF_Model__max_features': ['sqrt', 'log2', None],  
    'RF_Model__bootstrap': [True,],  
    'RF_Model__ccp_alpha': [0.0, 0.01, 0.1], 
    'RF_Model__max_samples': [None, 0.5, 0.75]  
}
# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=Base_RF_Model_Pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best score found: ", grid_search.best_score_)


### Grid Search Metrics For Random Forest Model

In [21]:
# Create Model Pipeline
Base_RF_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('Feature_Selection', Selection),
    ('RF_Model', RandomForestClassifier(random_state=42,
                                       
                                       ccp_alpha=  0.0,
                                       criterion ='entropy',
                                       max_depth= 20,
                                       max_features= 'sqrt',
                                       min_samples_leaf= 2,
                                       min_samples_split= 2,
                                       n_estimators= 100,
                                       bootstrap= False,
                                       max_samples= None,
))  
])

# Perform cross-validation
RF_CV_accuracy = cross_val_score(Base_RF_Model_Pipeline, X, Y, cv=10, scoring='accuracy').mean()
RF_CV_predictions = cross_val_predict(Base_RF_Model_Pipeline, X, Y, cv=10)

# Calculate recall, precision, and F1 score for each fold
RF_CV_recall = recall_score(Y, RF_CV_predictions, average=None).mean()
RF_CV_precision = precision_score(Y, RF_CV_predictions, average=None).mean()
RF_CV_f1_score = f1_score(Y, RF_CV_predictions, average=None).mean()

# Display the metrics
print("Accuracy:", RF_CV_accuracy)
print("Recall:", RF_CV_recall)
print("Precision:", RF_CV_precision)
print("F1 Score:", RF_CV_f1_score)

Accuracy: 0.5569727891156462
Recall: 0.45467674532940017
Precision: 0.4513094190140845
F1 Score: 0.45243897270723854


In [35]:
# ??RandomForestClassifier

In [13]:
# # Define the parameter grid, prefixing each parameter with the name of the pipeline step followed by two underscores
# param_grid = {
#     'GBC_Model__loss': ['deviance', 'exponential'],
#     'GBC_Model__learning_rate': [0.01, 0.1, 0.5],
#     'GBC_Model__n_estimators': [50, 100, 200],
#     'GBC_Model__subsample': [0.5, 0.75, 1.0],
#     'GBC_Model__criterion': ['friedman_mse', 'mse', 'mae'],
#     'GBC_Model__min_samples_split': [2, 5, 10],
#     'GBC_Model__min_samples_leaf': [1, 2, 4],
#     'GBC_Model__max_depth': [3, 5, 10],
#     'GBC_Model__min_impurity_decrease': [0.0, 0.1, 0.2],
#     'GBC_Model__max_features': ['auto', 'sqrt', 'log2', None],
#     'GBC_Model__warm_start': [True, False]
# }

# # Create GridSearchCV
# grid_search = GridSearchCV(estimator=Base_GBC_Model_Pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

# # Get the best parameters
# best_params = grid_search.best_params_

# print("Best parameters found:")
# print(best_params)

In [14]:
# ??RandomForestClassifier

In [15]:
# from sklearn.model_selection import GridSearchCV

# # Define the parameter grid to use for the grid search
# param_grid = {
#     'RF_Model__n_estimators': [10, 20,30,50,75, 100,150, 200],  # Number of trees in the forest
#     'RF_Model__criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
#     'RF_Model__max_depth': [None,5, 10,15, 20],  # Maximum depth of the tree
#     'RF_Model__min_samples_split': [2, 5, 10, 15,],  # Minimum number of samples required to split an internal node
#     'RF_Model__min_samples_leaf': [1,3, 5],  # Minimum number of samples required to be at a leaf node
#     'RF_Model__max_features': ['sqrt', 'log2', None],  # Number of features to consider when looking for the best split
#     # Note: For max_features, None means that max_features=n_features
#     'RF_Model__bootstrap': [True, False],  # Whether bootstrap samples are used when building trees
#     'RF_Model__ccp_alpha': [0.0, 0.01, 0.1],  # Complexity parameter used for Minimal Cost-Complexity Pruning
#     'RF_Model__max_samples': [None,0.25, 0.5, 0.75]  # If bootstrap is True, the number of samples to draw from X to train each base estimator
# }

# # Create the GridSearchCV object
# grid_search = GridSearchCV(estimator=Base_RF_Model_Pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# # Assuming X_train and y_train are already defined
# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

# # Print the best parameters and the best score
# print("Best parameters found: ", grid_search.best_params_)
# print("Best score found: ", grid_search.best_score_)


In [16]:
# #Create Pipeline to be used in Gradient Boosting Classfier Model
# Base_GBC_Model_Pipeline = Pipeline(steps=[
#     ('Preprocessor', Preprocessor),
#     ('GBC_Model',GradientBoostingClassifier())])

# # Perform cross-validation
# GBC_CV_accuracy = cross_val_score(Base_GBC_Model_Pipeline, X, Y, cv=5, scoring='accuracy').mean()
# GBC_CV_predictions = cross_val_predict(Base_GBC_Model_Pipeline, X, Y, cv=5)

# # Calculate recall, precision, and F1 score for each fold
# GBC_CV_recall = recall_score(Y, GBC_CV_predictions, average=None).mean()
# GBC_CV_precision = precision_score(Y, GBC_CV_predictions, average=None).mean()
# GBC_CV_f1_score = f1_score(Y, GBC_CV_predictions, average=None).mean()

# # Display the metrics
# print("Accuracy:", GBC_CV_accuracy)
# print("Recall:", GBC_CV_recall)
# print("Precision:", GBC_CV_precision)
# print("F1 Score:", GBC_CV_f1_score)


In [17]:
#Create Pipeline to be used in Gradient Boosting Classfier Model
Base_GBC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('GBC_Model',GradientBoostingClassifier(
    learning_rate=0.1,
    n_estimators=100,
    subsample=.75,
    criterion='friedman_mse',
    min_samples_split=10,
    min_samples_leaf=4,
    max_depth=5,
    min_impurity_decrease=0.1,
    random_state=42,
    max_features='sqrt',
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
   ))])

# Perform cross-validation
GBC_CV_accuracy = cross_val_score(Base_GBC_Model_Pipeline, X, Y, cv=5, scoring='accuracy').mean()
GBC_CV_predictions = cross_val_predict(Base_GBC_Model_Pipeline, X, Y, cv=5)

# Calculate recall, precision, and F1 score for each fold
GBC_CV_recall = recall_score(Y, GBC_CV_predictions, average=None).mean()
GBC_CV_precision = precision_score(Y, GBC_CV_predictions, average=None).mean()
GBC_CV_f1_score = f1_score(Y, GBC_CV_predictions, average=None).mean()

# Display the metrics
print("Accuracy:", GBC_CV_accuracy)
print("Recall:", GBC_CV_recall)
print("Precision:", GBC_CV_precision)
print("F1 Score:", GBC_CV_f1_score)

Accuracy: 0.5629939862542955
Recall: 0.4670907079646017
Precision: 0.4654887218045113
F1 Score: 0.466013109566001


In [18]:
from sklearn.model_selection import cross_validate, StratifiedKFold

# Define StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define scoring metrics
scoring = ['accuracy', 'recall', 'precision', 'f1']

# Perform cross-validation
cv_results = cross_validate(Base_GBC_Model_Pipeline, X, Y, cv=skf, scoring=scoring)

# Calculate the mean of each metric
accuracy_mean = cv_results['test_accuracy'].mean()
recall_mean = cv_results['test_recall'].mean()
precision_mean = cv_results['test_precision'].mean()
f1_mean = cv_results['test_f1'].mean()

print("StratifiedKFold CV Results:")
print(f"Accuracy: {accuracy_mean}")
print(f"Recall: {recall_mean}")
print(f"Precision: {precision_mean}")
print(f"F1 Score: {f1_mean}")


StratifiedKFold CV Results:
Accuracy: 0.6833118556701031
Recall: 0.8967954345917472
Precision: 0.7205529209160536
F1 Score: 0.7990249871289621


# Tuned Models 

In [78]:
#Create Pipeline to be used in Gradient Boosting Classfier Model
Base_GBC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('GBC_Model',GradientBoostingClassifier(
    learning_rate=0.1,
    n_estimators=100,
    subsample=.75,
    criterion='friedman_mse',
    min_samples_split=10,
    min_samples_leaf=4,
    max_depth=5,
    min_impurity_decrease=0.1,
    random_state=42,
    max_features='sqrt',
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
   ))])

#Fit the training data 
Base_GBC_Model_Pipeline.fit(X_train, y_train)

# Perform cross-validation
GBC_predictions = Base_GBC_Model_Pipeline.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, GBC_predictions)
print(f"\nTest Accuracy: {accuracy}\n")

# Generate  and Print a classification report
report = classification_report(y_test, GBC_predictions, target_names=["Loss Trades", "Win Trades"]) 
print("Classification Report:\n", report)


Test Accuracy: 0.75

Classification Report:
               precision    recall  f1-score   support

 Loss Trades       0.67      0.15      0.25        13
  Win Trades       0.76      0.97      0.85        35

    accuracy                           0.75        48
   macro avg       0.71      0.56      0.55        48
weighted avg       0.73      0.75      0.69        48



## Summary and Key Takeaways

### Performance Overview
- **Test Accuracy**: 75%.

### Insights from Classification Report
- **'Win Trades' (Positive Class)**:
  - Precision: ~76%
  - Recall: ~97%
  - F1-score: ~0.85
- **'Loss Trades' (Negative Class)**:
  - Precision: ~67%
  - Recall: ~15%
  - F1-score: ~0.25

### Key Takeaways
1. **Strong Performance on Majority Class**:
   - The model is highly effective in predicting 'Win Trades', with high precision and recall. This indicates strong performance for the majority class.
2. **Challenges with Minority Class**:
   - Despite a reasonable precision, the recall for 'Loss Trades' is very low, suggesting that the model struggles to identify this class correctly. The low recall for 'Loss Trades' needs addressing to balance the model's performance across classes.
3. **Potential for Model Tuning**:
   - Improving the model's ability to detect 'Loss Trades' could involve tuning hyperparameters, exploring different model architectures, or employing techniques aimed at balancing class representation.
4. **Metrics Indicating Room for Improvement**:
   - While the overall accuracy is good, the macro averages for precision, recall, and F1-score suggest that the performance across classes is uneven, reinforcing the need for model refinement to ensure fairness and robustness.

This summary pinpoints where the model excels and where improvements are necessary, especially in handling the minority class more effectively.


In [19]:
from sklearn.model_selection import ShuffleSplit

# Define ShuffleSplit
ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Perform cross-validation
cv_results = cross_validate(Base_GBC_Model_Pipeline, X, Y, cv=ss, scoring=scoring)

# Calculate the mean of each metric
accuracy_mean = cv_results['test_accuracy'].mean()
recall_mean = cv_results['test_recall'].mean()
precision_mean = cv_results['test_precision'].mean()
f1_mean = cv_results['test_f1'].mean()

print("ShuffleSplit CV Results:")
print(f"Accuracy: {accuracy_mean}")
print(f"Recall: {recall_mean}")
print(f"Precision: {precision_mean}")
print(f"F1 Score: {f1_mean}")


In [20]:
import numpy as np

In [21]:
# import numpy as np

# # Given values
# values = [0.01, 0.1, 0.5]

# # Using list comprehension with numpy.array to recreate the list
# list_from_numpy = [np.array(values)[i] for i in range(len(values))]

# print(list_from_numpy)


In [22]:
# # Define the parameter grid, prefixing each parameter with the name of the pipeline step followed by two underscores
# param_grid = {
#     'GBC_Model__loss': ['deviance', 'exponential'],
#     'GBC_Model__learning_rate': [0.01, 0.1, 0.5],
#     'GBC_Model__n_estimators': [50, 100, 200],
#     'GBC_Model__subsample': [0.5, 0.75, 1.0],
#     'GBC_Model__criterion': ['friedman_mse', 'mse', 'mae'],
#     'GBC_Model__min_samples_split': [2, 5, 10],
#     'GBC_Model__min_samples_leaf': [1, 2, 4],
#     'GBC_Model__max_depth': [3, 5, 10],
#     'GBC_Model__min_impurity_decrease': [0.0, 0.1, 0.2],
#     'GBC_Model__max_features': ['auto', 'sqrt', 'log2', None],
#     'GBC_Model__warm_start': [True, False]
# }

# # Create GridSearchCV
# grid_search = GridSearchCV(estimator=Base_GBC_Model_Pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

# # Get the best parameters
# best_params = grid_search.best_params_

# print("Best parameters found:")
# print(best_params)

In [23]:
# grid = GridSearchCV(Base_GBC_Model_Pipeline, params, cv=5, scoring='accuracy')
# grid.fit(X,Y)

In [24]:
# ??GradientBoostingClassifier

In [25]:
# Best parameters found:
# {'GBC_Model__criterion': 'friedman_mse',
#  'GBC_Model__learning_rate': 0.1, 
#  'GBC_Model__loss': 'exponential',
#  'GBC_Model__max_depth': 5,
#  'GBC_Model__max_features': 'sqrt', 
#  'GBC_Model__min_impurity_decrease': 0.1,
#  'GBC_Model__min_samples_leaf': 4, 
#  'GBC_Model__min_samples_split': 10,
#  'GBC_Model__n_estimators': 100,
#  'GBC_Model__subsample': 0.75,
#  'GBC_Model__warm_start': False}

### Base Model Summary 

In [ ]:
#Create a dataframe to score metric scores
Classifier_Scores = pd.DataFrame()

In [ ]:
# Create a DataFrame to store these values
metrics_df = pd.DataFrame({
    'Logistic Regression': [LR_Accuracy, LR_f_score, LR_recall_, LR_precision],
    'Descision Tree' : [DT_Accuracy, DT_f_score, DT_recall_, DT_precision],
    'Support Vector Machine' : [SVC_Accuracy, SVC_f_score, SVC_recall_, SVC_precision],
    'Random Forest' : [RF_Accuracy, RF_f_score, RF_recall_, RF_precision],
    'Gradient Boosting' : [GBC_Accuracy, GBC_f_score, GBC_recall_, GBC_precision],
    'ADA Boost ' : [ADAC_Accuracy, ADAC_f_score, ADAC_recall_, ADAC_precision],
}, index=['Accuracy','F1 Score', 'Recall', 'Precision'])

# Transpose the DataFrame 
metrics_df = metrics_df.transpose()

#rename the index 
metrics_df.index.name='Model Name'

metrics_df.reset_index(inplace=True)

# # Display the DataFrame
print(metrics_df)

In [ ]:
#Sort the df based on each metric 
accuracy_sorted_df = metrics_df.sort_values('Accuracy',ascending=False)
f1_sorted_df = metrics_df.sort_values('F1 Score',ascending=False)
recall_sorted_df = metrics_df.sort_values('Recall',ascending=False)
precision_sorted_df = metrics_df.sort_values('Precision',ascending=False)

#Create Figure 
fig, axs= plt.subplots(1,4, figsize=(24,10))

fig.suptitle("Base Model Performance",fontsize=16)

#Plot Accuracy in AXS[0,0]
bar_accuracy = axs[0].bar(accuracy_sorted_df['Model Name'], accuracy_sorted_df['Accuracy'])
axs[0].set_title('Accuracy By Classifiers')
axs[0].set_ylabel('Percentage %')
axs[0].set_xticklabels(accuracy_sorted_df['Model Name'], rotation=45)
for bar in bar_accuracy:
    height = bar.get_height()
    axs[0].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}%', ha='center', va='bottom')

#Plot Precision in AXS[1[]
bar_precision = axs[1].bar(precision_sorted_df['Model Name'], precision_sorted_df['Precision'])
axs[1].set_title('Precision By Classifiers')
axs[1].set_ylabel('Percentage %')
axs[1].set_xticklabels(precision_sorted_df['Model Name'], rotation=45)
for bar in bar_precision:
    height = bar.get_height()
    axs[1].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}%', ha='center', va='bottom')

#Plot Recall in AXS[2]
bar_recall = axs[2].bar(recall_sorted_df['Model Name'], recall_sorted_df['Recall'])
axs[2].set_title('Recall By Classifiers')
axs[2].set_ylabel('Percentage %')
axs[2].set_xticklabels(recall_sorted_df['Model Name'], rotation=45)
for bar in bar_recall:
    height = bar.get_height()
    axs[2].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}%', ha='center', va='bottom')
    
#Plot F1 Score in AXS[3]
bar_fscore = axs[3].bar(f1_sorted_df['Model Name'], f1_sorted_df['F1 Score'])
axs[3].set_title('F1 Score By Classifiers')
axs[3].set_ylabel('Percentage %')
axs[3].set_xticklabels(f1_sorted_df['Model Name'], rotation=45)
for bar in bar_fscore:
    height = bar.get_height()
    axs[3].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}%', ha='center', va='bottom')
    
plt.tight_layout()


### There might be an issue between the Recall and Precision value of Logistic Regression - do not delete this until investigate further 

### Feature Importance

In [ ]:
#Retrived the Feature Selection Step 
feature_selection = Base_RF_Model_Pipeline.named_steps['Feature_Selection']

#Retrieve the trained model 
trained_RF_Model = Base_RF_Model_Pipeline.named_steps['RF_Model']

feature_importances = trained_RF_Model.feature_importances_

# Get the indices of selected features
selected_feature_indices = feature_selection.get_support(indices=True)

# Extract feature names
all_feature_names = Preprocessor.get_feature_names_out()
selected_feature_names = [all_feature_names[i] for i in selected_feature_indices]

# Combine feature names with their importances
features_and_importances = zip(selected_feature_names, feature_importances)

# Sort features by importance
sorted_features_and_importances = sorted(features_and_importances, key=lambda x: x[1], reverse=True)


# Create a DataFrame from the features and importances
feature_importance_df = pd.DataFrame(sorted_features_and_importances, columns=['Feature', 'Importance'])

# Display the DataFrame
feature_importance_df.head(20)

### Reference: Concepts To Understand 

#### Accuracy: 
                - Measures the overall correctness of the classifier. It's calculated by dividing the number of correctly classifier samples by the total number of samples. 
                - Accuracy tells us how often the classifier correctly predicted whether a trade is a successful or not compared to the total number of trades.
                - An Accuracy of .76 means that classifier correctly predicted the trade status for approximately 76% of the time. 
                
#### Recall:
             - Also known as sensitivity or true positive, measures  the ability of the classifier to correctly identify all positive instances. 
             - In the context of a trade status classifier, recall tells us how often the classifier correctly identified out of all successful trades.
             - A recall of approximately .795 means that the classifier correctly identified 79.5% of the successful trades. 
#### Precision 
              - Measures the ability of the classifier to correctly identify only relevant instances among all instances it has classified as positive. 
              - In the context of a trade status classifier, precision measures the proportion of correctly predicted successful trades out of all successful trades.
              - A precision of approximately .933 means that about 93.3% of the trades predicted as successful by the classifier were actually successful. 
#### F1 Score
                - Is a metric that combines precision and recall into a single value - it is the harmonic mean between precision and recall. 
                - Precision measures the proportion of correctly predicted positive cases among all predicted cases.
                - While Recall, measures the proportion of correctly predicted positive cases among all actual positive cases.
                - F1 Score provides a balance between precision and recall, it is especially useful when there is an uneven class distribution.
                - In the context of a trade status classifier, an F1 Score of .859 indicate a good balance between precision and recall predicting whether trades were successful or not. 